---
title: "Robust Linear Programs - Modelling Discrete Failures"
author: "Vahram Poghosyan"
date: "2022-01-23"
categories: ["Optimization"]
image: "introduction_to_optimization.png"
format:
  html:
    code-fold: true
jupyter: python3
include-after-body:
  text: |
    <script type="application/javascript" src="../../javascript/light-dark.js"></script>

---

# Model

We are faced with the task of modeling a scenario in which at most $k$ of the total $n$ workers, machines, sensors, or other system components can fail. The task is to minimize the amount of system components needed, thereby minimizing cost, subject to certain known and robust constraints.

We will assume the linear cost function to be $c^Tx$, and the known constraints to be $Ax \leq b$. The robust constraint is $a_R^Tx \leq b_R, \ \ \forall a_R \in D_k$ where $b_R$ is a known vector, and $D_k$ is the following interval uncertainty set with an additional combinatorial constraint: 

$$
D_k = \{ a : a_i \in [\hat a_i - \delta_i, \hat a_i + \delta_i] \wedge \textrm{at least $n-k$ of the $a_i$'s exactly equal $\hat a_i$}\}
$$

In $D_k$, we can think of each $\hat a_i$ as the spec at which the $i$-th component should operate, and the $\delta_i$'s as the $i$-th component's deviation from this spec. Thus, $D_k$ models the discrete failures scenario exactly...

# Formulating as an Optimization Problem

Let's attempt to formulate this problem as a robust LP.

So far we have

$$
\begin{cases}
min: c^Tx
\\
s.t.: \begin{aligned} &Ax \leq b
\\ 
&a_R^Tx \leq b_R \ \ \forall a_r \in D_k
\end{aligned}
\end{cases}
$$

Which is, of course, not in standard form. 

For a fixed $x$, $a_R^Tx \leq b_R \ \ \forall a_r \ \ \implies \ \ a_{R \ max}^Tx \leq b_R$. But since we don't know the value of $a_{R \ max}$, we can formulate an inner optimization problem for which it is the optimal value.

So we can state the following equivalent optimization problem

$$
\begin{cases}
min: c^Tx
\\
s.t.: Ax \leq b
\\
\begin{cases}
max_{a_R}: &a_R^Tx 
\\
s.t.: &a_R \in D_k
\end{cases} \leq b_R
\end{cases}
$$

## Formulating the Inner Problem as a Linear Program 

Let's focus on the inner problem

$$
\begin{cases}
max_{a_R}: &a_R^Tx 
\\
s.t.: &a_R \in D_k
\end{cases} \leq b_R
$$

Our strategy now is to expand the constraint set $D_k$.

To that end, let's introduce slack variables $-1 \leq z_i \leq 1 \ \ \forall i$, which represent the direction of each component's deviation from its spec. We can now rewrite the objective as: 

$$
\begin{aligned}
a_R^Tx & = \sum a_ix_i 
\\
& = \sum (\hat a_i + z_i\delta_i)x_i 
\\
& = \sum \hat a_ix_i + \sum \delta_iz_ix_i
\end{aligned}
$$

So the optimization problem, which is now in the variables $z_i$, becomes

$$
\begin{cases}
max_{z}: \sum \hat a_ix_i + \sum \delta_iz_ix_i
\\
s.t.: \begin{aligned} &-1 \leq z_i \leq 1 \ \ \forall i
\\
& \textrm{at most $k$ of the $z_i \ne 0$}
\end{aligned}
\end{cases}
$$

We still have the combinatorial constraint 'at most $k$ of the $z_i \ne 0$,' which makes this into a mixed optimization problem... 

We need to massage this problem more to bring it to standard form.

Note that we're dealing with a problem of maximization. In the objective, $\sum \hat a_ix_i$ is fixed by virtue of the $\hat a_i$'s being fixed by the given $D_k$ and the $x_i$'s being fixed by the outer optimization problem. Note that the $\delta_i$'s are also fixed by $D_k$. Therefore, what would maximize the objective is each term of $\sum \delta_i z_i x_i$ contributing positively to the sum. 

This happens when $z_i$ and $x_i$ have the same sign $\forall i$. 
That is, their product $z_ix_i$ takes values in $[0, |x_i|]$. 

The remaining cases can be disposed of without changing the optimal value of the optimization problem.

Rewriting the problem, we have

$$
\begin{cases}
max_{z}: \sum \hat a_ix_i + \sum \delta_iz_i|x_i|
\\
s.t.: \begin{aligned} &0 \leq z_i \leq 1 \ \ \forall i
\\
& \textrm{at most $k$ of the $z_i \ne 0$}
\end{aligned}
\end{cases} \dagger
$$

::: {.callout-tip title="Note" appearance="minimal" collapse="false"}
Note that $|x_i|$ does not make the objective non-linear because the $x_i$'s are fixed values, and not the decision variables.
:::

### Relaxing the Combinatorial Constraint with a Continuous Constraint

We will relax the combinatorial constraint 'at most $k$ of the $z_i \ne 0$' by replacing it with $\sum z_i \leq k$.

Although this is a relaxation of the constraint, we will show that it makes no difference within the context of preserving the optimization problem. That is, it does not affect the optimal value of the problem.


First and foremost, it's easy to see that 'at most $k$ of the $z_i \ne 0$' $\implies$ $\sum z_i \leq k$ since each $z_i \in [0,1]$. 

We claim the converse is true as well, given that we restrict our attention to the optimal solution to the above LP $\dagger$. That is 'at most $k$ of the $z_i \ne 0$' $\impliedby$ $\sum z_i \leq k$.

This claim is true by the geometry of linear programs. An optimal solution to the LP can only occur at an extreme point, and those are defined exactly by $n$ independent active constraints. 

In the above LP, $0 \leq z_i \leq 1 \ \ \forall i$ represent a set of $2n$ independent constraints, and $\sum z_i \leq k$ is just one additional constraint.

If all of the $n$ active constraints come from $0 \leq z_i \leq 1 \ \ \forall i$, then since a given $z_i$ cannot simultaneously be $0$ and $1$ the $z_i$'s of the optimal solution must take integral values (that is, either $0$ or $1$ and nothing in between). 

In the general case, at least $n-1$ constraints must come from $0 \leq z_i \leq 1 \ \ \forall i$, which implies at least $n-1$ of the $z_i$'s take integral values and the remaining active constraint is $\sum z_i = k$. But $n$ numbers, of which $n-1$ are integers, cannot add up to an integer value $k$ unless the remaining number is also an integer. So, once again we have that all the $z_i$'s are integral valued. 

Then $\sum z_i \leq k$ $\implies$ at most $k$ of the $z_i = 1$ $\implies$ 'at most $k$ of the $z_i \ne 0$' as was the claim. 

This leaves us with the inner optimization problem

$$
\begin{cases}
max_{z}: \sum \hat a_ix_i + \sum \delta_iz_i|x_i|
\\
s.t.: \begin{aligned} &0 \leq z_i \leq 1 \ \ \forall i
\\
& \sum z_i \leq k
\end{aligned}
\end{cases}
$$

which is finally a linear program.

## Putting the Inner and Outer Problems Together

The combined optimization problem becomes

$$
\begin{cases}
min: c^Tx
\\
s.t.: Ax \leq b
\\
\begin{cases}
max_{z}: \sum \hat a_ix_i + \sum \delta_iz_i|x_i|
\\
s.t.: \begin{aligned} &0 \leq z_i \leq 1 \ \ \forall i
\\
& \sum z_i \leq k
\end{aligned}
\end{cases} \leq b_R
\end{cases}
$$

This is, of course, still not a linear program. Firstly, it's a mixture between minimization and maximization. Secondly, since the variables are $x_i$, and $z_i$, the term $\sum \delta_iz_ix_i$ is not linear in the decision variables. Thirdly, $|x_i|$ is not linear in $x_i$.

We can address these issues one by one... 

### Taking the Dual of the Inner

We can turn the inner maximization problem to an inner minimization problem by taking its dual. As we know, by LP-duality (otherwise known as [strong duality](https://en.wikipedia.org/wiki/Strong_duality)) this does not affect the optimal value of the problem. 

The overall problem becomes

$$
\begin{cases}
min: c^Tx
\\
s.t.: Ax \leq b
\\
\begin{cases}
min_{\lambda}: \sum \hat a_ix_i + \sum \lambda_i + \lambda_0k
\\
s.t.: \begin{aligned} &\lambda_0 + \lambda_i \geq \delta_i|x_i| \ \ \forall i
\\
& \lambda \geq 0
\end{aligned}
\end{cases} \leq b_R
\end{cases}
$$


Flattening the problems, since both are now minimization, we arrive at the following

$$
\begin{cases}
min_{x,\lambda}: c^Tx
\\
s.t.: \begin{aligned} &Ax \leq b
\\
&\sum \hat a_ix_i + \sum \lambda_i + \lambda_0k \leq b_R
\\
&\lambda_0 + \lambda_i \geq \delta_i|x_i| \ \ \forall i
\\
& \lambda \geq 0
\end{aligned}
\end{cases}
$$

### Linearizing the Absolute Value Constraints

This is almost a linear program, except for the fact that $|x_i|$'s are nonlinear terms in the constraint. The last step is to split these constraints into corresponding pairs of linear constraints.

For each $i$, 

$$
\begin{aligned} \lambda_0 + \lambda_i \geq \delta_i|x_i| &\implies -\lambda_0 - \lambda_i \leq \delta_ix_i \leq \lambda_0 + \lambda_i  \\ &\implies \begin{cases} \lambda_0 + \lambda_i &\geq \delta_ix_i \\ &\textrm{and} \\ \lambda_0 + \lambda_i &\geq -\delta_ix_i \end{cases}\end{aligned}
$$

So, the final problem, which is a linear program in every right, is 

$$
\begin{cases}
min_{x,\lambda}: c^Tx
\\
s.t.: \begin{aligned} &Ax \leq b
\\
&\sum \hat a_ix_i + \sum \lambda_i + \lambda_0k \leq b_R
\\
&\lambda_0 + \lambda_i \geq \delta_ix_i \ \ \forall i
\\
&\lambda_0 + \lambda_i \geq -\delta_ix_i \ \ \forall i
\\
& \lambda \geq 0
\end{aligned}
\end{cases}
$$

# Conclusion

Using the geometry of linear programs and LP duality, we were able to sidestep the complexity of a robust problem with a combinatorial constraint by formulating it as a linear program which can be solved by a number of fast algorithms such as the [Simplex Algorithm](https://en.wikipedia.org/wiki/Simplex_algorithm) or the [Interior Point Method](https://en.wikipedia.org/wiki/Interior-point_method). 

This shows the versatility of linear programs in addressing a variety of interesting mixed optimization problems.